### This code has been developed and run using the biodesign_3.7 kernel on the jprime.lbl.gov server. It uses the cplex library for running the MOMA optimization

## Imports

In [2]:
import collections as col
import os
import random
import re
import sys
import warnings
from shutil import copyfile
from enum import Enum
from typing import NewType, Dict, List, Any, OrderedDict, Counter

import cobra
import numpy as np
import pandas as pd
from cobra.exceptions import OptimizationError, Infeasible

## Global parameters

In [84]:
# Type annotations
Filename = NewType('Filename', str)

# Enumerations
class Omics(Enum):
    """Enumeration with supported omics data types."""
    PROTEOMICS = 0
    TRANSCRIPTOMICS = 1
    METABOLOMICS = 2

    def __str__(self):
        return f'{str(self.name).lower()}'


# Constants
UNIPROT_URL = '''https://www.uniprot.org/uploadlists/'''
CTS_URL = '''https://cts.fiehnlab.ucdavis.edu/rest/convert/'''
# HOST NAME
HOST_NAME: str = 'ropacus' 
# TODO: Move some constants to variables by program arguments
DATA_FILE_PATH: Filename = Filename('data')
# Output file path
OUTPUT_FILE_PATH: Filename = Filename('data/output')
# INCHIKEY_TO_CID_MAP_FILE_PATH: mapping file path to map inchikey to cids
INCHIKEY_TO_CID_MAP_FILE_PATH: Filename = Filename('mapping') 
# MODEL_FILENAME: Filename = Filename('iECIAI39_1322.xml')  # E. coli
MODEL_FILENAME: Filename = Filename('reannotated_base_v3.sbml')  # R. opacus
# TODO: rename traininig file name to design file names and paths
# Training file name
TRAINING_FILE_NAME: Filename = Filename('')
TRAINING_FILE_PATH: Filename = Filename('')
# Start time and stop time
TIMESTART: float = 0.0
TIMESTOP: float = 8.0
NUMPOINTS: int = 9
# number of reactions and instances
NUM_REACTIONS: int = None
NUM_INSTANCES: int = None

# NOTE: user input to the program
BIOMASS_REACTION_ID: str = 'BIOMASS_Ec_iJO1366_core_53p95M'  # E. coli
REACTION_ID: str = 'biomass_target'  # R. opacus
# REACTION_ID: str = 'SRC_C00185_e'  # R. opacus
GENE_IDS_DBS: List[str] = ['kegg.genes']  # R. opacus
# GENE_IDS_DBS: List[str] = ['uniprot', 'goa', 'ncbigi']  # E. coli
UNITS: Dict[Omics, str] = {
    Omics.PROTEOMICS: 'proteins/cell',
    Omics.TRANSCRIPTOMICS: "FPKM",
    Omics.METABOLOMICS: "mg/L"
}
# Fix the flux value to a particular value
LOWER_BOUND: int = -15
UPPER_BOUND: int = -15

## FUNCTION A
### [Fluxes,extMetabolitesCon, OD]   = getFluxTimeSeries(GSM, InitialExtMetabolitesCon)

In [128]:
def get_flux_time_series(model, ext_metabolites, parameters):
    '''
    Generate fluxes and OD
    '''
    
   
    # time steps to calculate the biomass production for
    t0 = parameters['timestart']
    tf = parameters['timestop']
    points = parameters['numpoints']
    tspan, delt = np.linspace(t0, tf, points, dtype='float64', retstep=True)

    # panda series containing OD values for the timepionts
    cell = pd.Series(index=tspan)
    # NOTE: Do we to define this by the user?
    cell0 = user_params['initial_OD'] # in gDW/L
    cell[t0] = cell0

    # getting external metabolites and their initial concentrations
    met_names = []
    initial_concentrations = []
    for met, init_conc in ext_metabolites.items():
        met_names.append(met)
        initial_concentrations.append(init_conc)

    # Dataframe containing external metabolites
    subs = pd.DataFrame(index=tspan, columns=met_names)
    # initial concentrations of external metabolites
    subs.loc[t0] = initial_concentrations
    
    # fluxomics data for all timepoints
    fluxomics_timeseries = {}
    
    # series to contain the biomass optimization solution across timepoints
    sol_time_wild = pd.Series(index=tspan)

    # exchange reactions and external metabolites dictionary
    subs_ext = {r.id: r.reactants[0].id for r in model.exchanges if r.reactants[0].id in met_names}

    # NOTE: put the body of the for loop inside a function
    for t in tspan:
        # Not changing the model but adding constraints for each time point
        with model:
            # why do we set volume to one? Is it arbitrary?
            volume = 1.0
            for k, v in subs_ext.items():
                # Set global reactions bounds (in addition to local)
                # set the lower bound to the maximum of the lower bound of the metabolite in the model 
                # and the existing amount of the metabolite so that it doe not go below the lower bound
                model.reactions.get_by_id(k).lower_bound = max(model.reactions.get_by_id(k).lower_bound, -subs.loc[t,v]*volume/cell[t]/delt)
            
            # get optimized solution later used to calculate metabolite concentrations and fluxes in model
            sol_t = model.optimize()
            
            # Step 2: storing the solution for each timepoint which are going to be reference solutions for moma
            sol_time_wild[t] = sol_t
            
            # Step 3: Calculate mu where mu = solution(biomass)
            mu = sol_t[BIOMASS_REACTION_ID]
            print(t, sol_t.status, mu)
            
            # Step 4: Calculate ext. metabolite concentrations for next time point t+delta:
#             iso = 'EX_isoprenol_e'
            if sol_t.status == 'optimal' and mu > 1e-6:
                # Calculating next time point's OD
                cell[t+delt] = cell[t]*np.exp(mu*delt)
                # Calculating external substrate concentrations for next time point
                for k, v in subs_ext.items():
                    subs.loc[t+delt,v] = max(subs.loc[t,v]-sol_t[k]/mu*cell[t]*(1-np.exp(mu*delt)),0.0)     
#                     if k == iso:
#                         if sol_t[k] > 0:
#                             # Calculating isoprenol concentration for next time point
#                             subs.loc[t+delt,v] = subs.loc[t,v]-sol_t[k]/mu*cell[t]*(1-np.exp(mu*delt))
#                         else:
#                             subs.loc[0:t, v] = 0
#                             subs.loc[t+delt,v] = subs.loc[t,v]-sol_t[k]/mu*cell[t]*(1-np.exp(mu*delt))
#                     else:
#                         subs.loc[t+delt,v] = max(subs.loc[t,v]-sol_t[k]/mu*cell[t]*(1-np.exp(mu*delt)),0.0)     
                
            else:
                cell[t+delt] = cell[t]
                for k, v in subs_ext.items():
                    subs.loc[t+delt,v] = subs.loc[t,v]

            # get fluxomics data from the GSM and
            # appending the fluxomics data for each timepoint into a Series
            fluxomics_timeseries[t] = sol_t.fluxes.to_dict()
            
    return fluxomics_timeseries, cell, subs, subs_ext, sol_time_wild

## Function C
### (Trans,Prot,Met) = getMultiomics(Fluxes, GSM)

In [139]:
def read_pubchem_id_file():
    inchikey_to_cid = {}
    filename = f'{INCHIKEY_TO_CID_MAP_FILE_PATH}/inchikey_to_cid.txt'
    with open(filename, 'r') as fh:
            try:
                line = fh.readline()
                while line:
                    # checking to ignore inchikey records with no cid mappings
                    if (len(line.split()) > 1):
                        inchikey_to_cid[line.split()[0]] = 'CID:'+line.split()[1]
                    else:
                        inchikey_to_cid[line.strip()] = None

                    line = fh.readline()
            # NOTE: propagated exception, raise
            except Exception as ex:
                print("Error in reading file!")
                print(ex)

    return inchikey_to_cid


def get_proteomics_transcriptomics_data(model, solution):
    """

    :param model:
    :param solution:
    :param condition:
    :return:
    """

    # pre-determined linear constant (NOTE: Allow user to set this via parameter)
    # DISCUSS!!
    k = 0.8
    q = 0.06

    proteomics = {}
    transcriptomics = {}

    rxnIDs = solution.fluxes.keys()
    for rxnId in rxnIDs:
        reaction = model.reactions.get_by_id(rxnId)
        for gene in list(reaction.genes):

            # this will ignore all the reactions that does not have the gene.annotation property
            # DISCUSS!!
            if gene.annotation:
                if 'uniprot' not in gene.annotation:
                    if 'goa' in gene.annotation:
                        protein_id = gene.annotation['goa']
                    else:
                        break
                else:
                    protein_id = gene.annotation['uniprot'][0]
                
                # add random noise wjhich is 5 percent of the signal
                noiseSigma = 0.05 * solution.fluxes[rxnId]/k;
                noise = noiseSigma*np.random.randn();
                proteomics[protein_id] = (solution.fluxes[rxnId]/k) + noise

            # create transcriptomics dict
            noiseSigma = 0.05 * proteomics[protein_id]/q;
            noise = noiseSigma*np.random.randn();
            transcriptomics[gene.id] = (proteomics[protein_id]/q) + noise

    return proteomics, transcriptomics

# NOTE: Work on this
def get_metabolomics_data(model):
    """

    :param model:
    :param condition:
    :return:
    """
    metabolomics = {}
    # get metabolites

    # read the inchikey to pubchem ids mapping file
    inchikey_to_cid = {}
    inchikey_to_cid = read_pubchem_id_file()
    
    for met in model.metabolites:
        # get associated reactions involving metabolite
        num_reactions_with_flux = 0
        for reaction in list(met.reactions):
            # get dictionary of associated metabolites and their concentrations
            # average fluxes
            
            # if there is an inchikey ID for the metabolite
            if 'inchi_key' in met.annotation:
                # if it is a list get the first element
                if type(met.annotation['inchi_key']) is list:
                    inchi_key = met.annotation['inchi_key'][0]
                else:
                    inchi_key = met.annotation['inchi_key']
                
                # if the inchikey for the metabolite is in the mapping dict
                if inchi_key in inchikey_to_cid.keys():
                    # if the CID is not in the metabolomics dict keys AND the mappned value is not None and the reactions flux is not 0
                    if (inchikey_to_cid[inchi_key] not in metabolomics.keys()) and (inchikey_to_cid[inchi_key] is not None) and reaction.flux != 0.0:
                            metabolomics[inchikey_to_cid[inchi_key]] = abs(reaction.flux)
                            num_reactions_with_flux  += 1
                    elif (inchikey_to_cid[inchi_key] is not None) and reaction.flux != 0.0:
                            metabolomics[inchikey_to_cid[inchi_key]] += abs(reaction.flux)
                            num_reactions_with_flux  += 1

        # check if inchi_key attribite present else ignore metabolite
        if 'inchi_key' in met.annotation.keys() and inchi_key in inchikey_to_cid.keys() and inchikey_to_cid[inchi_key] is not None:
            metabolomics[inchikey_to_cid[inchi_key]]/=num_reactions_with_flux

    return metabolomics

def get_multiomics(model, solution):
    """

    :param model: cobra model object
    :param solution: solution for the model optimization using cobra
    :param data_type: defines the type of -omics data to generate (all by default)
    :return:
    """

    proteomics = {}
    transcriptomics = {}
    fluxomics = {}
    metabolomics = {}

    proteomics, transcriptomics = get_proteomics_transcriptomics_data(model, solution)

    metabolomics = get_metabolomics_data(model)

    return (proteomics, transcriptomics, metabolomics)



## Function B
### BEFluxes = getBEFluxes(Fluxes, Change, GSM)

In [135]:
def getBEFluxes(fluxes_timeseries, design_df, model, sol_time_wild, tspan, delt, cell, subs, subs_ext, n_reactions=None, n_instances=None):
    model.solver = 'cplex'
    reaction_names = list(design_df.columns[1:])

    # Calculating the number of reactions that should be modified (n_genes) and 
    # number of strains for which isoprenol concentration should be estimated 
    # mod_reax
    if n_reactions is None:
        n_reactions = design_df.shape[1] - 1
    if n_instances is None:
        n_instances = design_df.shape[0] - 1
        
    # series to contain the biomass optimization solution across timepoints
    sol_time_moma = pd.Series(index=tspan)

    volume = 1.0
    # For each strain
    for i in range(0,n_instances):
        # At each time point
        for t in tspan:
            # Adding constraints to the model at each time point for each strain without globally changing the model
            with model:
                for k, v in subs_ext.items():
                    model.reactions.get_by_id(k).lower_bound = max(model.reactions.get_by_id(k).lower_bound,
                                                            -subs.loc[t,v]*volume/cell[t]/delt)

                # Adding the fluxed modifications for chosen reactions
                for reaction in reaction_names:
                    reaction_constraint = model.problem.Constraint(model.reactions.get_by_id(reaction).flux_expression, 
                                                lb = sol_time_wild[t][reaction]*design_df.iloc[i,1],
                                                ub = sol_time_wild[t][reaction]*design_df.iloc[i,1])
                    model.add_cons_vars(reaction_constraint)

                # Reference solution calculated for each time point in above cell for wild type
                sol1 = sol_time_wild[t]

                # Moma solution for each time point
                sol2 = cobra.flux_analysis.moma(model, solution=sol1, linear=True) # we changed the linear=True for fadster calculation
                
                # saving the moma solutions across timepoints
                sol_time_moma[t] = sol2
                
    return sol_time_moma
                
                

## Function D
### ExtMetabolites =  Integrate(Fluxes)

In [183]:
# def integrate_fluxes(ext_metabolites, solution_timeseries, model):
#     '''
#     Returns the tortal fluxes across all timepoints for all external metabolites
#     '''
#     extra_metabolite_fluxes = pd.Series(0, index=ext_metabolites)
#     print(extra_metabolite_fluxes)
#     model_metabolite_names = [met.id for met in model.metabolites]
#     model_metabolites = [met for met in model.metabolites]
    
#     for metabolite in ext_metabolites:
#         if metabolite in model_metabolite_names:
#             reactions = list(model.metabolites.reactions)
#         for reaction in reactions:
            
#         for t, solution in solution_timeseries.items():
#             print(metabolite)
#             print(solution)
#             extra_metabolite_fluxes[metabolite] += solution[metabolite]
    
#     return extra_metabolite_fluxes

# sol_time_moma
# e_metabolites = list(user_params['ext_metabolites'].keys())
# extra_metabolite_fluxes = integrate_fluxes(e_metabolites, sol_time_moma)
# extra_metabolite_fluxes
# model.metabolites[0].id

'10fthf_c'

## Add isopentenol pathway to model

In [165]:
def add_isopentenol_pathway(model, sce):
    # Load S. cerevisiae model
    #     sce = cobra.io.load_json_model(f'data/{cerevisiae_modelfile}')
    
    # Add mevalonate pathway reactions from S. cerevisiae model
    for x in ['HMGCOAS','HMGCOAR','MEVK1','DPMVD']:
        r = sce.reactions.get_by_id(x).copy()
        r.gene_reaction_rule = ''
        model.add_reaction(r)
    
    # Update gene names
    model.reactions.get_by_id('HMGCOAS').gene_reaction_rule = 'HMGS'
    model.reactions.get_by_id('HMGCOAR').gene_reaction_rule = 'HMGR'
    model.reactions.get_by_id('MEVK1').gene_reaction_rule = 'MK'
    model.reactions.get_by_id('DPMVD').gene_reaction_rule = 'PMD'
    
    # Add IP to model
    m = model.metabolites.ipdp_c.copy()
    m.id = 'ipmp_c'
    m.name = 'Isopentenyl monophosphate'
    m.formula = 'C5H9O4P'
    m.charge = -2
    model.add_metabolites([m])
    # Update PMD reaction to convert mev-5p to IP
    model.reactions.get_by_id('DPMVD').id = 'PMD'
    model.reactions.get_by_id('PMD').add_metabolites({'5dpmev_c': 1.0, '5pmev_c': -1.0,
                                                      'ipdp_c': -1.0, 'ipmp_c': 1.0})
    # Add isoprenol (isopentenol)
    m = model.metabolites.ipmp_c.copy()
    m.id = 'isoprenol_c'
    m.name = 'Isopentenol'
    m.formula = 'C5H10O'
    m.charge = 0
    model.add_metabolites([m])
    # Add phosphatase reaction by AphA
    r = model.reactions.CHLabcpp.copy()
    r.id = 'IPMPP'
    r.name = 'Isopentenyl monophosphate phosphatase'
    r.gene_reaction_rule = 'AphA'
    model.add_reactions([r])
    r.add_metabolites({'chol_p': 1.0, 'atp_c': 1.0, 'chol_c': -1.0, 'adp_c': -1.0, 'h_c': -1.0, 'ipmp_c': -1.0, 'isoprenol_c': 1.0})
    
    # Add periplasmic and extracellular isoprenol
    m = model.metabolites.isoprenol_c.copy()
    m.id = 'isoprenol_p'
    m.compartment = 'p'
    model.add_metabolites([m])
    m = model.metabolites.isoprenol_c.copy()
    m.id = 'isoprenol_e'
    m.compartment = 'e'
    model.add_metabolites([m])
    # Add periplasmic and extracellular transport reactions
    r = model.reactions.ETOHtrpp.copy()
    r.id = 'IPtrpp'
    r.name = 'Isopentenol reversible transport via diffusion (periplasm)'
    r.gene_reaction_rule = ''
    model.add_reactions([r])
    r.add_metabolites({'etoh_p': 1.0, 'etoh_c': -1.0, 'isoprenol_p': -1.0, 'isoprenol_c': 1.0})
    r = model.reactions.ETOHtex.copy()
    r.id = 'IPtex'
    r.name = 'Isopentenol transport via diffusion (extracellular to periplasm)'
    r.gene_reaction_rule = ''
    model.add_reactions([r])
    r.add_metabolites({'etoh_e': 1.0, 'etoh_p': -1.0, 'isoprenol_e': -1.0, 'isoprenol_p': 1.0})
    # Add a boundary reaction
    r = model.reactions.EX_etoh_e.copy()
    r.id = 'EX_isoprenol_e'
    r.name = 'Isopentenol exchange'
    r.gene_reaction_rule = ''
    model.add_reactions([r])
    r.add_metabolites({'etoh_e': 1.0, 'isoprenol_e': -1.0})
    
    # Write model to files
    outputfilename = user_params['modelfile'].split('.')[0] + '_IPP.json'
    cobra.io.save_json_model(model, f'data/{outputfilename}')
    
    return model

## Check if model has IPP pathway

In [155]:
def model_has_IPP_pathway(model):
    reaction_list = ['HMGCOAS','HMGCOAR','MEVK1','PMD','IPMPP','IPtrpp','IPtex','EX_isoprenol_e']
    model_reactions = [r.id for r in model.reactions]
    for reac in reaction_list:
        if reac not in model_reactions:
            return False
    return True

# model = cobra.io.load_json_model('iJO1366.json')
# model_has_IPP_pathway(model)
        

False

## User parameters

In [158]:
user_params = {
    'host': 'ecoli', # ecoli or ropacus
    'modelfile': 'iJO1366_MVA.json',
    'cerevisiae_modelfile': 'iMM904.json',
    'modelfilepath': 'sample_files',
    'timestart': 0.0,
    'timestop': 8.0,
    'numpoints': 9,
    'designfile': 'training_data_8genes.csv',
    'designfilepath': 'data',
    'numreactions': 8,
    'numinstances': 5,
    'ext_metabolites': {
        'glc__D_e': 22.203,
        'nh4_e': 18.695,
        'pi_e': 69.454,
        'so4_e': 2.0,
        'mg2_e': 2.0,
        'k_e': 21.883,
        'na1_e': 103.7,
        'cl_e': 27.25,
        'isoprenol_e': 0.0
    },
    'initial_OD': 0.01
}

# comp = ['glc__D_e', 'nh4_e', 'pi_e', 'so4_e', 'mg2_e', 'k_e', 'na1_e', 'cl_e', 'EX_isoprenol_e']
# initial_concentrations = [22.203, 18.695, 69.454, 2.0, 2.0, 21.883, 103.7, 27.25] 

## Call functions
### Function A

In [168]:
file_name = user_params['modelfile']
file_name = 'iJO1366.json'
model = cobra.io.load_json_model(file_name)

# checking if model has IPP pathway
if not model_has_IPP_pathway(model):
    cerevisiae_model = cobra.io.load_json_model(f'data/{user_params["cerevisiae_modelfile"]}')
    model = add_isopentenol_pathway(model, cerevisiae_model)

# adding minimum flux constriants for isopentenol and formate
iso = 'EX_isoprenol_e'
iso_cons = model.problem.Constraint(model.reactions.EX_isoprenol_e.flux_expression,
                        lb = 0.20)
model.add_cons_vars(iso_cons)
for_cons = model.problem.Constraint(model.reactions.EX_for_e.flux_expression,
                        lb = 0.10)
model.add_cons_vars(for_cons)

In [ ]:
# get the fluxomics time series, OD, externalk metabolites and soltion for the time series
parameters = {
    'timestart': user_params['timestart'],
    'timestop': user_params['timestop'],
    'numpoints': user_params['numpoints']
}
fluxomics_timeseries, od, ext_metabolites, exchange_reactions, solution_timeseries = get_flux_time_series(model, user_params['ext_metabolites'], parameters)

In [167]:
solution_timeseries, od, ext_metabolites

(0.0         <Solution 0.958 at 0x7fc58e835c50>
 1.0         <Solution 0.958 at 0x7fc588eaea10>
 2.0         <Solution 0.958 at 0x7fc588eaefd0>
 3.0         <Solution 0.958 at 0x7fc588eae9d0>
 4.0         <Solution 0.958 at 0x7fc588f3e690>
 5.0         <Solution 0.448 at 0x7fc588f3e410>
 6.0    <Solution infeasible at 0x7fc4e502f150>
 7.0    <Solution infeasible at 0x7fc4e4fcdad0>
 8.0    <Solution infeasible at 0x7fc588f3e710>
 dtype: object, 0.0    0.010000
 1.0    0.026062
 2.0    0.067924
 3.0    0.177026
 4.0    0.461371
 5.0    1.202438
 6.0    1.881649
 7.0    1.881649
 8.0    1.881649
 9.0    1.881649
 dtype: float64,     glc__D_e    nh4_e     pi_e    so4_e    mg2_e      k_e  na1_e     cl_e  \
 0.0   22.203   18.695   69.454        2        2   21.883  103.7    27.25   
 1.0  22.0353  18.5215  69.4385  1.99595  1.99986  21.8799  103.7  27.2499   
 2.0  21.5983  18.0694  69.3981  1.98539   1.9995  21.8717  103.7  27.2497   
 3.0  20.4593   16.891  69.2929  1.95787  1.99855  21.8

## Function B

In [137]:
# we need the solution for the wild type for maximum biomass production
design_df = pd.read_csv(f'{user_params["designfilepath"]}/{user_params["designfile"]}')
sol_time_moma = getBEFluxes(fluxomics_timeseries, design_df, model, solution_timeseries, tspan, delt, od, ext_metabolites, exchange_reactions, user_params['numreactions'], user_params['numinstances'])

In [138]:
sol_time_moma

0.0        <Solution 85.527 at 0x7fc595851b10>
1.0        <Solution 85.527 at 0x7fc588ebd150>
2.0        <Solution 85.527 at 0x7fc58dfeafd0>
3.0        <Solution 85.527 at 0x7fc589c64b50>
4.0        <Solution 85.527 at 0x7fc589bf9a90>
5.0       <Solution 127.366 at 0x7fc588949610>
6.0    <Solution infeasible at 0x7fc5895f0990>
7.0    <Solution infeasible at 0x7fc58df1e210>
8.0    <Solution infeasible at 0x7fc58dea0ad0>
dtype: object

## Function C

In [ ]:
tspan, delt = np.linspace(user_params['timestart'], user_params['timestop'], user_params['numpoints'], dtype='float64', retstep=True)
proteomics_timeseries = {}
transcriptomics_timeseries = {}
metabolomics_timeseries = {}
for t, solution in solution_timeseries.items():
    proteomics_timeseries[t], transcriptomics_timeseries[t], metabolomics_timeseries[t] = get_multiomics(model, solution)

## Function D

In [143]:
integrate_fluxes(ext_metabolites)

glc__D_e        113.541483
nh4_e            91.812369
pi_e            685.500041
so4_e            17.636435
mg2_e            19.918703
k_e             217.000765
na1_e          1037.000000
cl_e            272.451222
isoprenol_e       2.602829
dtype: float64